<a href="https://colab.research.google.com/github/lawgorithm/sql_practice/blob/main/Y_on_Y_Growth_Rate_%5BWayfair_SQL_Interview_Question%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://datalemur.com/questions/yoy-growth-rate

In [ ]:
%%sql
WITH yearly_spend_cte AS (
  SELECT
    EXTRACT(YEAR FROM transaction_date) AS yr,
    product_id,
    spend AS curr_year_spend,
    # LAG() function allows you to access data of the previous row from the current row
    LAG(spend) OVER (
      # Sort of acts a a group by. I think mainly this means LAG() evaluated
      # against the first row for the product_id will return NULL, rather than
      # the last spend value for a different product
      PARTITION BY product_id
      # It's crucial to do the order by correctly, since that determines the
      # semantics of what the "last" row means
      ORDER BY
        product_id,
        EXTRACT(YEAR FROM transaction_date)) AS prev_year_spend
  FROM user_transactions
)

SELECT
  yr,
  product_id,
  curr_year_spend,
  prev_year_spend,
  ROUND(100 *
    (curr_year_spend - prev_year_spend)
    / prev_year_spend
  , 2) AS yoy_rate
FROM yearly_spend_cte;

In [ ]:
# Note: I think this is mostly correct but the website won't accept it because
# it drops rows where the prior year is NULL
%%sql
WITH first_query AS (
SELECT
  EXTRACT(YEAR FROM transaction_date) as yr,
  product_id,
  SUM(spend) as spend
FROM user_transactions as this_year
GROUP BY yr, product_id
)
SELECT
  a.product_id,
  a.yr as this_year,
  b.yr as last_year,
  a.spend as curr_year_spend,
  b.spend as prev_year_spend,
  ROUND(100 * (a.spend / b.spend - 1), 2) as yoy_rate
FROM
  first_query a
  LEFT JOIN
  first_query b
  ON (a.product_id = b.product_id)
WHERE
  b.yr = a.yr - 1
ORDER BY product_id, this_year ASC
;